<a href="https://colab.research.google.com/github/vasyl-d/vasyl-d/blob/main/TestRO0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Настройки

In [45]:
import time
import json
import requests
import math
import pandas as pd

#@title api_key for remonline and Date: from / to:
API_KEY = "237a1231fe9b4a13b66e5389598b9d85" #@param {type:"string"}
#@title Date fields
DATE_FROM = "2022-09-26" #@param {type:"date"}
DATE_TO = "2022-9-30" #@param {type:"date"}

# API_KEY = "237a1231fe9b4a13b66e5389598b9d85"
RO_PATH = "https://api.remonline.app"

class DictObj:
    '''класс для преобразования json в обьект'''
    def __init__(self, in_dict:dict):
       	if isinstance(in_dict, dict):
            for key, val in in_dict.items():
                if isinstance(val, (list, tuple)):
                    setattr(self, key, [DictObj(x) if isinstance(x, dict) else x for x in val])
                else:
                    setattr(self, key, DictObj(val) if isinstance(val, dict) else val)
        else:
            assert False, 'must be dict in parm'


class RemOnline():
    __instance_id = None
    token = None
    token_lifetime = None
    def __new__(cls):
        if cls.__instance_id == None:
             cls.__instance_id = super().__new__(cls)
        return cls.__instance_id 

    def __init__(self) -> None:
        self.tokenize()

    def tokenize(self):
        if self.token == None or self.is_token_invalid():
            self.token = self.set_token()
            self.token_lifetime = time.time()+550
        return False if self.token == None else True

    def is_token_invalid(self):
        return True if self.token_lifetime <= time.time() else False

    def set_token(self):
        url = RO_PATH+'/token/new'
        token = None
        payload={'api_key': API_KEY}
        files=[

        ]
        headers = {}
        try:
            response = requests.request("POST", url, headers=headers, data=payload, files=files)
            response.raise_for_status()
            data = json.loads(response.text)
            token = data["token"]
        except Exception as error:
            print(error)
        
        return token


class GetResurs:
    '''класс для итерации по Гет запросам, пагинация по 50 єлементов'''
    
    def __init__(self, ro:RemOnline, resurs:str, filterstr:str=''):
        self.ro = ro
        self.resurs = resurs
        self.res = None
        self.filterstr = filterstr
        self.page_len = 50

    def __iter__(self):
        self.limit = 1
        self.page = 1
        return self

    def __next__(self):
        if self.page <= self.limit:
            headers = {}
            payload = {}
            files ={}
            if not self.ro.tokenize():
                return self.res 
                
            url = RO_PATH + self.resurs + '?token='+str(self.ro.token)
            if self.filterstr !='':
                url +='&'+ self.filterstr
            if self.page >1 :
                url += '&page='+str(self.page)
            try:
                response = requests.request("GET", url, data=payload, headers=headers, files=files)
                response.raise_for_status()
                data = json.loads(response.text)
                self.limit = math.ceil(int(data["count"])/self.page_len)
                self.res = data['data']
                # self.res = DictObj(response.json()) #- так можно вернуть страницу как объект
            except Exception as error:
                print(error)
            self.page += 1
            return self.res
        else:
            raise StopIteration                    

ro = RemOnline()
tf = tuple(map(int, DATE_FROM.split("-") + [0]*6))
tt = (tuple(map(int, DATE_TO.split("-") + [0]*6)))
tday1 = round(time.mktime(tf))*1000
tday2 = round(time.mktime(tt))*1000

flt = "created_at[]="+str(tday1)+"&created_at[]="+str(tday2)
ord = GetResurs(ro, '/order/', flt)
myiter = iter(ord)
#возьмем только 1 cраницу
page = next(myiter)
df = pd.DataFrame(page)
df['modified_at'] = pd.to_datetime(df['modified_at'])
df.head()
# for order in page.data:
#     print(order.id, order.id_label, order.client.name, order.status.name, time.localtime(order.created_at/1000)[:3])

,id,finance_transaction,modified_at,uuid,status,prepared_at,created_at,done_at,client,kindof_good,...,overdue,status_overdue,parts,operations,attachments,order_type,created_by_id,closed_by_id,ad_campaign,status_deadline
0,38410094,{'pk': 108656445},1970-01-01 00:27:44.442432,4cae8b94-af84-4d1c-8b8e-dfcd1ec69404,"{'id': 309139, 'name': 'Оплатил', 'color': '#9...",1664164802000,1664164802000,1.664442e+12,"{'id': 17642123, 'name': 'Сервисный центр', 'e...",,...,False,False,[],[],[],"{'id': 85892, 'name': '💀 Churn'}",94618,94618.0,{},NaN
1,38410095,{'pk': 108656446},1970-01-01 00:27:44.458588,e79dc4e5-18aa-420f-a4cc-355cbcb214fc,"{'id': 309139, 'name': 'Оплатил', 'color': '#9...",1664164803000,1664164803000,1.664459e+12,"{'id': 18916394, 'name': 'Smart Service | Ремо...",,...,False,False,[],[],[],"{'id': 85892, 'name': '💀 Churn'}",94618,94618.0,{},NaN
2,38410096,{'pk': 108656447},1970-01-01 00:27:44.391426,f2696add-a2f4-4c5d-a9e4-5b74c7b695b9,"{'id': 309139, 'name': 'Оплатил', 'color': '#9...",1664164803000,1664164803000,1.664391e+12,"{'id': 17268699, 'name': 'БСК', 'email': 'pchi...",,...,False,False,[],[],[],"{'id': 85892, 'name': '💀 Churn'}",94618,94618.0,{},NaN
3,38410097,{'pk': 108656448},1970-01-01 00:27:44.164803,2701fc13-e56f-4c43-a170-ede510df8030,"{'id': 309138, 'name': 'Новый', 'color': '#428...",1664164803000,1664164803000,NaN,"{'id': 19574862, 'name': 'IMMO PRO SERVICE', '...",,...,False,True,[],[],[],"{'id': 85892, 'name': '💀 Churn'}",94618,NaN,{},1.664215e+12
4,38410099,{'pk': 108656450},1970-01-01 00:27:44.451671,47ec5f18-4793-4e96-8057-347e2726d3dd,"{'id': 309139, 'name': 'Оплатил', 'color': '#9...",1664164804000,1664164804000,1.664452e+12,"{'id': 18339647, 'name': 'SIA ""Office Tech Ser...",,...,False,False,[],[],[],"{'id': 85892, 'name': '💀 Churn'}",94618,94618.0,{},NaN


# Новый раздел